In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
from tqdm import tqdm
import shutil
from tqdm import tqdm


In [2]:
if(os.path.split(os.getcwd())[1] == "misc"):
    os.chdir("..")
print("Current Working Directory: {}".format(os.path.split(os.getcwd())[1]))

cuda = True

Current Working Directory: progettoVIPM


In [ ]:
# Extract one example from each class and put it in "unique_classes.csv"

file = np.loadtxt("train_small.csv", delimiter=",", dtype="str")
file[:,1] = file[:,1].astype("int")
unique_classes = np.unique(file[:,1], return_index=True)[0].astype("int")
unique_indexes = np.unique(file[:,1], return_index=True)[1]
holder = np.stack((unique_indexes, unique_classes), axis=-1)

holder = np.sort(holder, axis=0)
unique_filenames = file[holder[:,0]]


print(unique_filenames)
rows = ["{},{}".format(i, j) for i, j in unique_filenames] 
text = "\n".join(rows) 


with open('unique_classes.csv', 'w') as f: 
    f.write(text)

In [ ]:
# Separate all of the images classes from the labelled training set and isolate them in folders in parent folder "separated_training_set" 

file = np.loadtxt("train_small.csv", delimiter=",", dtype="str")
file[:,1] = file[:,1].astype("int")
print(file)
path = "./separated_training_set/"
for i in range(0,251):
    path_folder = path + str(i) 
    if not os.path.exists(path_folder):
        os.makedirs(path_folder)


for instance in tqdm(file):
    path_folder = path + instance[1]
    shutil.copy("./train_set/{}".format(instance[0]), path_folder)


In [48]:
# Extract a validation set from the labelled training set
# Note: running the code will overwrite the files of the generated split

validation_im_per_class = 4
path = "./separated_training_set/"
val_set = np.empty((validation_im_per_class*251,2)).astype(str)
train_set = np.empty(((20-validation_im_per_class)*251,2)).astype(str)
for i in range(0,251):
    path_folder = path + str(i)
    file_list = np.array(os.listdir(path_folder))
    start_class_index = i*(validation_im_per_class)
    end_class_index = start_class_index+validation_im_per_class
    random_extraction = np.random.permutation(20)
    chosen_images = file_list[random_extraction[:validation_im_per_class].tolist()]
    val_set[start_class_index:end_class_index, 0] = chosen_images[:]
    val_set[start_class_index:end_class_index, 1] = i

    start_class_index = i*(20-validation_im_per_class)
    end_class_index = start_class_index+(20-validation_im_per_class)
    chosen_images = file_list[random_extraction[validation_im_per_class:].tolist()]
    train_set[start_class_index:end_class_index, 0] = chosen_images[:]
    train_set[start_class_index:end_class_index, 1] = i

print(val_set.shape)
print(train_set.shape)

val_percentage = ((validation_im_per_class*251)/(251*20))*100
filename = "validation_set_{}%.csv".format(int(val_percentage))
np.savetxt(filename, val_set,  delimiter = ",", fmt='%s')

filename = "training_set_{}%.csv".format(100-int(val_percentage))
np.savetxt(filename, train_set,  delimiter = ",", fmt='%s')

(1004, 2)
(4016, 2)
